In [103]:
# этот скрипт делает запрос к openai api.

In [104]:
#%pip install numpy
#%pip install pandas
#%pip install python-dotenv

In [105]:
import base64
import requests
from dotenv import load_dotenv
import os
import json

# Загрузка переменных из файла .env
load_dotenv()

# Получение значения переменной OPENAI_API_KEY
api_key = os.getenv('OPENAI_API_KEY')

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [106]:
# Path to your image
image_path = "img/1T.png"

# Getting the base64 string
base64_image = encode_image(image_path)

In [107]:
payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "system", 
      "content": "You are a professional quantitative analyst specializing in financial market analysis and price" 
              + " forecasting. You always spend a few sentences explaining background context, assumptions, and" 
              + " step-by-step thinking BEFORE you try to answer a question to ensure accuracy. You conduct a"
              + " thorough analysis of the overall market situation before providing forecasts."
    },
    {
      "role": "user", 
      "content": [
        {
          "type": "text", 
          "text": "Could you please analyze the attached minute candlestick chart and provide forecasts for the next"
                + " 5 minutes. The main chart includes Bollinger Bands (period=20, color=black) and ATR"
                + " (period=10, color=red). The volume chart also includes RSI (period=14, color=black)."
                + " Below the volume chart is a panel with MACD (fast=12, slow=26, signal=9). Please MAKE SURE TO"
                + " INCLUDE the predicted CLOSING PRICE RANGE after the next 5 minutes as well as the PROBABILITIES of"
                + " price increase and decrease."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          },
        }
      ],
    }
  ],
  "max_tokens": 1000
}

# делаем первый запрос к API, в котором просим провести анализ графика.
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
j = response.json()
j

{'id': 'chatcmpl-9VRu50v1EHfXorAAIsV5EXgfcNNix',
 'object': 'chat.completion',
 'created': 1717282457,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "To provide an analysis of this minute candlestick chart, we'll consider the following indicators: Bollinger Bands, Average True Range (ATR), Relative Strength Index (RSI), and Moving Average Convergence Divergence (MACD).\n\n### Observations:\n1. **Candlestick Movement & Bollinger Bands:**\n   The price is currently moving near the lower Bollinger Band, indicating a possible oversold condition. The bands are relatively wide, which suggests increased volatility.\n\n2. **Average True Range (ATR):**\n   The ATR, shown in red, is moderately high, indicating substantial recent volatility.\n\n3. **Volume and RSI:**\n   The volume has shown a significant increase around 07:28, which was followed by a price drop. The RSI is around 30, close to the oversold threshold, suggesting that th

In [108]:
# эта ячейка берет ответ из предыдущей ячейки и вставляет его в маркдаун ячейку ниже, чтобы было удобно читать.

# Запустите следующий код в Jupyter Notebook, чтобы создать функцию, которая будет добавлять новую ячейку с содержимым:
from IPython.display import display, Markdown, Javascript

def create_markdown_cell(content):
    display(Markdown(content))

# Создайте функцию, которая удаляет следующую ячейку, если она имеет тип Markdown:
def delete_next_markdown_cell():
    display(Javascript("""
    var currentCell = Jupyter.notebook.get_selected_index();
    var nextCell = Jupyter.notebook.get_cell(currentCell + 1);
    if (nextCell && nextCell.cell_type === 'markdown') {
        Jupyter.notebook.delete_cell(currentCell + 1);
    }
    """))

# Извлечение значения поля content
content = j['choices'][0]['message']['content']

# Удаление следующей Markdown ячейки, если она существует
delete_next_markdown_cell()

# Создание новой markdown ячейки с содержимым
create_markdown_cell(content)

<IPython.core.display.Javascript object>

To provide an analysis of this minute candlestick chart, we'll consider the following indicators: Bollinger Bands, Average True Range (ATR), Relative Strength Index (RSI), and Moving Average Convergence Divergence (MACD).

### Observations:
1. **Candlestick Movement & Bollinger Bands:**
   The price is currently moving near the lower Bollinger Band, indicating a possible oversold condition. The bands are relatively wide, which suggests increased volatility.

2. **Average True Range (ATR):**
   The ATR, shown in red, is moderately high, indicating substantial recent volatility.

3. **Volume and RSI:**
   The volume has shown a significant increase around 07:28, which was followed by a price drop. The RSI is around 30, close to the oversold threshold, suggesting that the current price might be undervalued, with a potential for a rebound.

4. **MACD:**
   The MACD histogram is in negative territory, and the MACD line is below the signal line, indicating bearish momentum, but the gap seems to be narrowing.

### Forecasting the Next 5 Minutes:
Given these observations, let's predict the range of the closing price for the next 5 minutes and the probabilities of price movement.

### Predictions:
1. **Price Range Estimation:**
   - **Upside Potential:** Based on Bollinger Bands, the upper bound can be estimated near the middle Bollinger Band, which is approximately 67850 to 67900.
   - **Downside Potential:** The lower bound could revisit the lower Bollinger Band at around 67500.

2. **Probabilities:**
   - **Price Increase Probability (P↑):** Considering the RSI is near the oversold region, and the MACD is showing signs of potential convergence, the likelihood of a short-term reversal and modest rally is around 60%.
   - **Price Decrease Probability (P↓):** Despite the oversold signs, the overall trend observed on the MACD and the position relative to the lower Bollinger Band still signal a bearish sentiment. Hence, the probability of continued slight downward movement or stabilization is around 40%.

### Predicted Closing Price Range:
- Range: 67500 to 67900
- Probability of Increase: 60%
- Probability of Decrease: 40%

Please note that these predictions are based on historical technical patterns and don't account for unforeseen market events or news. Always consider multiple indicators and sources before making trading decisions.

In [109]:
content

"To provide an analysis of this minute candlestick chart, we'll consider the following indicators: Bollinger Bands, Average True Range (ATR), Relative Strength Index (RSI), and Moving Average Convergence Divergence (MACD).\n\n### Observations:\n1. **Candlestick Movement & Bollinger Bands:**\n   The price is currently moving near the lower Bollinger Band, indicating a possible oversold condition. The bands are relatively wide, which suggests increased volatility.\n\n2. **Average True Range (ATR):**\n   The ATR, shown in red, is moderately high, indicating substantial recent volatility.\n\n3. **Volume and RSI:**\n   The volume has shown a significant increase around 07:28, which was followed by a price drop. The RSI is around 30, close to the oversold threshold, suggesting that the current price might be undervalued, with a potential for a rebound.\n\n4. **MACD:**\n   The MACD histogram is in negative territory, and the MACD line is below the signal line, indicating bearish momentum, but

In [110]:
# в этой ячейке делаем второй запрос к API, в котором просим предыдущий ответ оформить в виде json.

payload = {
  "model": "gpt-3.5-turbo",# берем модель 3.5 турбо, т.к. она в 10 раз дешевле gpt-4o и с задачей справляется.
  "response_format": { "type": "json_object" },
  "messages": [
    {
      "role": "system", 
      "content": """You are an LLM assistant specialized in converting financial analysis text into structured JSON format.
 Your task is to extract key numerical values and probabilities from the provided analysis and format
 them into a JSON object with the following fields: 'h' for the predicted highest price, 'l' for the
 predicted lowest price, 'p_up' for the probability of price increase, 'p_down' for the probability
 of price decrease, and 'error' for any potential issues or discrepancies noted during extraction.
 Ensure the JSON output accurately reflects the information in the analysis, adheres to the following
 JSON Schema:

```
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "h": {
      "type": "number",
      "description": "Predicted highest price"
    },
    "l": {
      "type": "number",
      "description": "Predicted lowest price"
    },
    "p_up": {
      "type": "number",
      "description": "Probability of price increase",
      "minimum": 0,
      "maximum": 100
    },
    "p_down": {
      "type": "number",
      "description": "Probability of price decrease",
      "minimum": 0,
      "maximum": 100
    },
    "error": {
      "type": "string",
      "description": "Error message if any issue is found"
    }
  },
  "additionalProperties": false
}
```

Respond only in JSON format without any additional text, as your response will be processed automatically by a program."""
    },
    {
      "role": "user", 
      "content": [
        {
          "type": "text", 
          "text": content
        }
      ],
    }
  ],
  "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
j = response.json()
j

{'id': 'chatcmpl-9VRuFlvqCdkgHLXp0SGAK6HKZuFlM',
 'object': 'chat.completion',
 'created': 1717282467,
 'model': 'gpt-3.5-turbo-0125',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{\n  "h": 67900,\n  "l": 67500,\n  "p_up": 60,\n  "p_down": 40,\n  "error": ""\n}'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 879, 'completion_tokens': 39, 'total_tokens': 918},
 'system_fingerprint': None}

In [111]:
json_string = j['choices'][0]['message']['content']

# Преобразование строки JSON в объект Python (словарь)
data = json.loads(json_string)

# Чтение значения 'h'
data["h"], data['l'], data['p_up'], data['p_down'], data['error']

(67900, 67500, 60, 40, '')